# IOバウンド処理のマルチプロセスのテスト

### マルチプロセス、スレッドなし

In [ ]:
import time

from process_iobound_one_by_one import wrap_calc, calc
CPU_COUNT = 1
TIMES = 1000
N = 10
start = time.time()
job_args = [(i, N) for i in range(TIMES)]
# print(wrap_calc((2, 100)))
# print(calc(2, 100))
for result in map(wrap_calc, job_args):
    pass
    # print(result)
print(time.time()-start)


### concurrent.futuresによるIOバウンド処理

In [2]:
import time
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures


from  process_iobound_one_by_one import wrap_calc, calc
CPU_COUNT = 6
TIMES = 1000
N = 10

job_args = [(i, N) for i in range(TIMES)]
start = time.time()
with ThreadPoolExecutor(max_workers=CPU_COUNT) as executor:
    futures = {executor.submit(wrap_calc, args):args for args in job_args}
    for i, future in enumerate(concurrent.futures.as_completed(futures)):
        result = futures[future]
        # print(result)
        # print(future.result())
print(time.time()-start)


0.42957091331481934


### joblibマルチプロセスIOバウンド

In [1]:
import time
from joblib import Parallel, delayed

from process_iobound_one_by_one import wrap_calc, calc
CPU_COUNT = 6
TIMES = 1000
N = 10

job_args = [(i, N) for i in range(TIMES)]

start = time.time()
# print(wrap_calc((2, 100)))
# print(calc(2, 100))
for result in Parallel(n_jobs=CPU_COUNT)([delayed(wrap_calc)(args) for args in job_args]):
    pass
    # print(result)
print(time.time()-start)


1
2.1131396293640137


0.4から2.1程度で安定しない

### joblibスレッドでのIOバウンド処理

In [1]:
import time
from joblib import Parallel, delayed

from process_iobound_one_by_one import wrap_calc, calc
CPU_COUNT = 6
TIMES = 1000
N = 10

job_args = [(i, N) for i in range(TIMES)]

start = time.time()
# print(wrap_calc((2, 100)))
# print(calc(2, 100))
for result in Parallel(n_jobs=CPU_COUNT, backend='threading')([delayed(wrap_calc)(args) for args in job_args]):
    pass
    # print(result)
print(time.time()-start)


1
0.4501981735229492


#### concurrent.futureとそん色なし👍